# PBL Fase 6 — Entrega 2
## Comparativo: YOLO custom (Entrega 1) × YOLO padrão × CNN do zero

**Objetivo**: Reutilizar o mesmo dataset da Entrega 1 e comparar:
1. **YOLO customizada** (treinada na Entrega 1).
2. **YOLO padrão (baseline)** — ex.: `yolov5s`/`yolov8n` treinada do zero no *seu* dataset.
3. **CNN do zero** (classificação A vs B).



## 0) Preparação do ambiente (Colab)
- Conectar Google Drive
- Instalar dependências YOLOv5 e Ultralytics (YOLOv8)
- Configurar variáveis de caminho


## 1) Definição dos caminhos e parâmetros do dataset
Preencha de acordo com **a mesma estrutura da sua Entrega 1**.

- `DATASET_DIR`: pasta raiz contendo `images` e `labels` por split (YOLO) ou pastas por classe (classificação).
- `DATA_YAML`: arquivo `.yaml` do YOLO com `train`, `val`, (opcional `test`) e `names`.

> `DATA_YAML = "/content/drive/MyDrive/training-ia-test/barco/barco.yaml"`


## 2) Utilitários de benchmark
Funções auxiliares para medir tempo de **treinamento** e **inferência**, e consolidar métricas.


## 3) YOLO Custom (reuso da Entrega 1)
- Reutilize seus **pesos treinados** na Entrega 1 (ex.: `runs/train/expX/weights/best.pt`).
- Avalie no conjunto de **teste** e colete métricas (**mAP**, **precision**, **recall**, etc.).
- Meça **tempo de inferência** em um lote de imagens de teste.


## 4) YOLO Padrão (baseline) — Treino do zero
Treine um modelo leve (ex.: `yolov5s`) por **duas configurações de épocas** (30 e 60). Colete métricas de validação/teste e compare.


## 5) Preparação para CNN (classificação)
A CNN requer **pastas por classe** (ex.: `train/A`, `train/B`, `val/A`…). Se seu dataset está em formato YOLO (imagens + labels), usa-se uma regra simples:

- Para cada imagem, lê-se o arquivo `.txt` com `class_id x_center y_center width height`.
- A classe **majoritária na imagem** define o rótulo de **classificação da imagem**.
- Caso haja empate ou múltiplos objetos, você pode **definir manualmente** ou **pular a imagem**.

> Ajuste `YOLO_IMAGES_DIR` e `YOLO_LABELS_DIR` conforme sua estrutura.


## 6) CNN do zero (PyTorch)
Treina uma CNN simples para **classificar A vs B**. Coletamos **accuracy/precision/recall/F1** e **tempo de treino**. 


## 7) Consolidação e comparação
Monte uma tabela comparando **precisão**, **F1**, **tempo de treino**, **tempo de inferência** e, quando possível, **mAP** (YOLO). Registre observações qualitativas (facilidade de uso/integração, tamanho do modelo, etc.).


## 8) Análise crítica (Markdown)
- **Facilidade de uso/integração**: YOLOv5/YOLOv8 têm APIs prontas, require anotação bbox. CNN exige dataset por classe, mas é simples para *classificação pura*.
- **Precisão do modelo**: YOLO para **detecção** (mAP); CNN para **classificação** (accuracy/F1). Pode variar com balanceamento, qualidade e tamanho da base.
- **Tempo de treinamento/customização**: comparar 30 vs 60 épocas para ambas abordagens.
- **Tempo de inferência**: medir ms/imagem em GPU/CPU quando possível; YOLOv5s/YOLOv8n são rápidos.
- **Quando usar cada uma**: detecção (localizar + classificar) vs classificação (decidir qual classe global).

> **Limitações**: dataset pequeno (80 imagens) pode levar a overfitting.


In [ ]:

# === Monta o Google Drive ===
from google.colab import drive
drive.mount('/content/drive')

# === Clona YOLOv5 e instala dependências ===
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt -q

# === Instala Ultralytics (YOLOv8) opcionalmente ===
!pip install ultralytics -q

# === Imports comuns ===
import os, time, json, shutil, glob, re, math, random
from pathlib import Path
import numpy as np
import pandas as pd


In [ ]:

DATA_YAML = "/content/drive/MyDrive/training-ia-test/barco/barco.yaml"  
DATASET_DIR = "/content/drive/MyDrive/training-ia-test/barco"  

CLASS_NAMES = None  # ex.: ["barco", "nao_barco"]

# Tamanho de imagem
IMG_SIZE_YOLO = 640
IMG_SIZE_CNN  = 224

# Épocas para comparação
EPOCHS_LIST = [30, 60]  # conforme enunciado

In [ ]:

# Tenta ler os nomes das classes do YAML (modo simples)
def load_names_from_yaml(yaml_path):
    try:
        import yaml
        with open(yaml_path, 'r') as f:
            data = yaml.safe_load(f)
        if isinstance(data.get('names'), list):
            return [str(x) for x in data['names']]
        elif isinstance(data.get('names'), dict):
            # dict {id: name}
            return [data['names'][k] for k in sorted(data['names'])]
    except Exception as e:
        print("Não foi possível ler names do YAML:", e)
    return None

if CLASS_NAMES is None:
    try:
        from google.colab import drive  # sanity check for Colab
        names = load_names_from_yaml(DATA_YAML)
        if names:
            CLASS_NAMES = names
            print("Classes (do YAML):", CLASS_NAMES)
        else:
            CLASS_NAMES = ["classe_A", "classe_B"]
            print("Classes (default):", CLASS_NAMES)
    except:
        CLASS_NAMES = ["classe_A", "classe_B"]
        print("Classes (default):", CLASS_NAMES)

In [ ]:

import time
from contextlib import contextmanager

@contextmanager
def timer(label):
    start = time.time()
    yield
    end = time.time()
    print(f"[TIMER] {label}: {end - start:.2f} s")

def ms_per_image(seconds, n_images):
    return (seconds / max(1, n_images)) * 1000.0


In [ ]:

BEST_WEIGHTS = "{latest_run}/weights/best.pt"  

# Avaliação com yolov5 val.py
!python yolov5/val.py --weights "{BEST_WEIGHTS}" --data "{DATA_YAML}" --img {IMG_SIZE_YOLO} --task test --verbose --save-json

# Inferência em imagens de teste (para prints)
TEST_SOURCE = f"{DATASET_DIR}/test"
!python yolov5/detect.py --weights "{BEST_WEIGHTS}" --img {IMG_SIZE_YOLO} --conf 0.25 --source "{TEST_SOURCE}" --save-txt --save-conf



In [ ]:

import os, json, time

results_rows = []

for E in EPOCHS_LIST:
    print(f"\n=== Treinando YOLOv5s por {E} épocas ===")
    with timer(f"YOLOv5s treino {E} épocas"):
        !python yolov5/train.py --data "{DATA_YAML}" --weights yolov5s.pt --img {IMG_SIZE_YOLO} --epochs {E} --project yolov5/runs/train --name yolo_baseline_{E} --exist-ok
    
    # Validação no conjunto de teste
    with timer(f"YOLOv5s val (test) {E} épocas"):
        !python yolov5/val.py --weights "yolov5/runs/train/yolo_baseline_{E}/weights/best.pt" --data "{DATA_YAML}" --img {IMG_SIZE_YOLO} --task test --verbose --save-json

print("Treinos baseline concluídos. Verifique pastas em yolov5/runs/train/.")

In [ ]:

import os, shutil, glob
from collections import Counter
from pathlib import Path

YOLO_IMAGES = {
    "train": f"{DATASET_DIR}/images/train",
    "val":   f"{DATASET_DIR}/images/val",
    "test":  f"{DATASET_DIR}/images/test"
}
YOLO_LABELS = {
    "train": f"{DATASET_DIR}/labels/train",
    "val":   f"{DATASET_DIR}/labels/val",
    "test":  f"{DATASET_DIR}/labels/test"
}

# Saída para classificação
CLS_ROOT = "/content/classification_ds"
for split in ["train","val","test"]:
    for cname in CLASS_NAMES:
        os.makedirs(f"{CLS_ROOT}/{split}/{cname}", exist_ok=True)

def infer_image_label_from_yolo(txt_path):
    try:
        lines = Path(txt_path).read_text().strip().splitlines()
        if not lines:
            return None
        classes = [int(line.split()[0]) for line in lines if line.strip()]
        if not classes:
            return None
        maj = Counter(classes).most_common(1)[0][0]
        if maj < len(CLASS_NAMES):
            return CLASS_NAMES[maj]
    except Exception as e:
        return None
    return None

def build_classification_split(split):
    img_dir = YOLO_IMAGES[split]
    lbl_dir = YOLO_LABELS[split]
    images = glob.glob(os.path.join(img_dir, "*.*"))
    copied = 0
    for img in images:
        stem = Path(img).stem
        txt = os.path.join(lbl_dir, stem + ".txt")
        if not os.path.exists(txt):
            continue
        cname = infer_image_label_from_yolo(txt)
        if cname is None:
            continue
        dst = os.path.join(CLS_ROOT, split, cname, os.path.basename(img))
        shutil.copy2(img, dst)
        copied += 1
    print(f"[{split}] copiados {copied} arquivos.")

for sp in ["train","val","test"]:
    build_classification_split(sp)

print("Estrutura para classificação criada em:", CLS_ROOT)


In [ ]:

!pip install torchmetrics -q

import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchmetrics.classification import BinaryF1Score, BinaryPrecision, BinaryRecall

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Transforms
train_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE_CNN, IMG_SIZE_CNN)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])
eval_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE_CNN, IMG_SIZE_CNN)),
    transforms.ToTensor()
])

train_ds = datasets.ImageFolder(f"{CLS_ROOT}/train", transform=train_tf)
val_ds   = datasets.ImageFolder(f"{CLS_ROOT}/val",   transform=eval_tf)
test_ds  = datasets.ImageFolder(f"{CLS_ROOT}/test",  transform=eval_tf)

train_dl = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
val_dl   = DataLoader(val_ds,   batch_size=32, shuffle=False, num_workers=2, pin_memory=True)
test_dl  = DataLoader(test_ds,  batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

# CNN simples
class SimpleCNN(nn.Module):
    def __init__(self, n_classes=2):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*(IMG_SIZE_CNN//8)*(IMG_SIZE_CNN//8), 128), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(128, n_classes)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = SimpleCNN(n_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

def evaluate(loader):
    model.eval()
    total, correct = 0,0
    all_probs = []
    all_labels = []
    with torch.no_grad():
        for x,y in loader:
            x,y = x.to(device), y.to(device)
            logits = model(x)
            probs = torch.softmax(logits, dim=1)[:,1]
            preds = torch.argmax(logits, dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
            all_probs.append(probs.detach().cpu())
            all_labels.append(y.detach().cpu())
    import torch
    probs = torch.cat(all_probs)
    labels = torch.cat(all_labels)
    # Para binário, classe "1" é a segunda pasta em train_ds.classes
    f1 = BinaryF1Score().to('cpu')(probs, labels).item()
    prec = BinaryPrecision().to('cpu')(probs, labels).item()
    rec = BinaryRecall().to('cpu')(probs, labels).item()
    acc = correct/total if total>0 else 0
    return {"acc":acc, "f1":f1, "precision":prec, "recall":rec}

history = []
for E in EPOCHS_LIST:  # treinar duas variações (30 e 60)
    print(f"\n=== CNN treino por {E} épocas ===")
    model = SimpleCNN(n_classes=2).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    with timer(f"CNN treino {E} épocas"):
        for epoch in range(E):
            model.train()
            running = 0.0
            for x,y in train_dl:
                x,y = x.to(device), y.to(device)
                optimizer.zero_grad()
                logits = model(x)
                loss = criterion(logits, y)
                loss.backward()
                optimizer.step()
                running += loss.item()*y.size(0)
            if (epoch+1)%10==0 or epoch==0:
                val_metrics = evaluate(val_dl)
                print(f"Epoch {epoch+1}/{E} - Val: {val_metrics}")

    test_metrics = evaluate(test_dl)
    print(f"[CNN] Test ({E} épocas):", test_metrics)
    row = {"approach":"CNN", "epochs":E, **test_metrics}
    history.append(row)

pd.DataFrame(history)


In [ ]:

cols = ["approach","epochs","acc","f1","precision","recall","notes"]
df = pd.DataFrame(columns=cols)

# Exemplos (substitua pelos seus resultados reais)
df = pd.concat([
    pd.DataFrame([{"approach":"YOLO custom (E1)","epochs":"-", "acc":None,"f1":None,"precision":None,"recall":None,"notes":"mAP@0.5 ~ X; ver val.py"}]),
    pd.DataFrame([{"approach":"YOLO baseline","epochs":30, "acc":None,"f1":None,"precision":None,"recall":None,"notes":"mAP@0.5 ~ ?"}]),
    pd.DataFrame([{"approach":"YOLO baseline","epochs":60, "acc":None,"f1":None,"precision":None,"recall":None,"notes":"mAP@0.5 ~ ?"}]),
    pd.DataFrame([{"approach":"CNN","epochs":30, "acc":None,"f1":None,"precision":None,"recall":None,"notes":"preencher com teste"}]),
    pd.DataFrame([{"approach":"CNN","epochs":60, "acc":None,"f1":None,"precision":None,"recall":None,"notes":"preencher com teste"}]),
], ignore_index=True)

print("Preencha os campos com os números obtidos nas seções anteriores.")
df
